In [2]:
import pandas as pd
import gurobipy as gp
import numpy as np

In [3]:
data = pd.read_pickle('doctor_assignment_data.pkl')

In [4]:
ranks = data['ranks'].item()
schools = data['schools'].item()
capacity = data['capacity'].item()
m = data['m'].item()
M = data['M'].item()

In [34]:
model = gp.Model('doctor_assignment')
X = model.addVars(100, 12, vtype=gp.GRB.BINARY, name='doctor_assignment')
model.setObjective( gp.quicksum( ranks[i][j] * X[i,j] for i in range(100) for j in range(12)) / 100,
                   gp.GRB.MINIMIZE)

CAP = model.addConstrs( gp.quicksum(X[i,j] for i in range(100)) <= capacity[j] for j in range(12) )
ONE_JOB = model.addConstrs( gp.quicksum(X[i,j] for j in range(12)) == 1 for i in range(100))
SCHOOL_JOB_MIN = model.addConstrs(gp.quicksum(schools[i][k] * X[i,j] for i in range(100)) >= m[k][j]
                                  for j in range(12) for k in range(4) )
SCHOOL_JOB_MAX = model.addConstrs(gp.quicksum(schools[i][k] * X[i,j] for i in range(100)) <= M[k][j]
                                  for j in range(12) for k in range(4) )
TOP_5 = model.addConstr(gp.quicksum( schools[i][3] * X[i,j] for i in range(100) for j in range(5)) >= 10)
MIN_MAX_RANK = model.addConstrs( ranks[i][j] * X[i,j] <= 7 for i in range(100) for j in range(12))

In [35]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1409 rows, 1200 columns and 6110 nonzeros
Model fingerprint: 0x90f5ec6c
Variable types: 0 continuous, 1200 integer (1200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 1206 rows and 500 columns
Presolve time: 0.00s
Presolved: 203 rows, 700 columns, 2804 nonzeros
Variable types: 0 continuous, 700 integer (700 binary)

Root relaxation: objective 2.480000e+00, 263 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.4800000    2.48000  0.00%     -    0s

Explored 1 nodes (263 simplex iterations) in 0.02 seconds (0.01 work uni

In [17]:
X_result = np.array([v.X for v in X.values()], dtype=int).reshape(100,12)

for j in range(12):
    print('Field %d:' %j)
    for i in range(100):
        if X_result[i, j] == 1:
            print('Doctor %d ' %(i+1), end='')
    print('\n')

AttributeError: Unable to retrieve attribute 'X'

In [16]:
model.objVal

AttributeError: Unable to retrieve attribute 'objVal'